# 如何处理速率限制
:::info 前提条件
本指南假设您熟悉以下概念：- [聊天模型](/docs/concepts/chat_models)- [大型语言模型（LLMs）](/docs/concepts/text_llms):::

你可能会遇到因向模型提供商API发送过多请求而被限速的情况。
例如，当您运行多个并行查询以在测试数据集上对聊天模型进行基准测试时，可能会出现这种情况。
如果你正面临这种情况，可以使用限流器来帮助将你的请求速率与允许的速率相匹配通过API。
:::info 需要 ``langchain-core >= 0.2.24``
此功能已在 ``langchain-core == 0.2.24`` 版本中添加。请确保您的软件包已更新至最新版本。好的,我会以标准markdown格式进行翻译,只输出具体内容:

# 翻译助手使用指南

## 基本功能
1. 提供高质量的英中互译服务
2. 保持原文的markdown格式不变
3. 支持技术文档、商务文件等多种文本类型

## 使用说明
- 输入要翻译的英文文本
- 系统会自动识别并保留markdown标记
- 输出格式化的中文翻译结果

## 注意事项
* 请确保输入文本为英文
* 复杂格式可能会需要人工校对
* 专业术语翻译仅供参考

> 提示：对于长文档建议分段翻译以获得最佳效果

## 初始化一个限流器
Langchain 内置了一个内存中的速率限制器。该速率限制器是线程安全的，可以在同一进程中被多个线程共享使用。
提供的速率限制器仅能限制单位时间内的请求数量。若您还需基于请求大小进行限制，则该限制器将无法满足需求。请求的

In [4]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.1,  # <-- Super slow! We can only make a request once every 10 seconds!!
    check_every_n_seconds=0.1,  # Wake up every 100 ms to check whether allowed to make a request,
    max_bucket_size=10,  # Controls the maximum burst size.
)

## 选择模型
选择任意模型并通过 `rate_limiter` 属性将速率限制器传递给它。

In [2]:
import os
import time
from getpass import getpass

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass()


from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model_name="claude-3-opus-20240229", rate_limiter=rate_limiter)

让我们确认速率限制器是否正常工作。我们应该每10秒只能调用模型一次。

In [3]:
for _ in range(5):
    tic = time.time()
    model.invoke("hello")
    toc = time.time()
    print(toc - tic)

11.599073648452759
10.7502121925354
10.244257926940918
8.83088755607605
11.645203590393066
